Интернет-магазин «Кислород» хочет запустить новую функциональность. Руководство хочет, чтобы у пользователей была возможность самостоятельно вносить изменения в описания товаров, как в вики-сообществах. Другими словами, клиенты предлагают свои правки и комментируют изменения других. Интернет-площадке необходим инструмент, способный искать токсичные комментарии и отправлять их на модерацию. 

Нам необходимо обучить модель так, чтобы она могла классифицировать комментарии на позитивные и негативные. Мы располагаем набором данных с разметкой о токсичности правок.

Цель проекта - модель со значением метрики качества *F1* не меньше 0.75. 

# 1. Подготовка

Загрузим необходимые инструменты.

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.linear_model import LogisticRegression
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import f1_score, make_scorer, confusion_matrix, accuracy_score, recall_score, precision_score
from nltk.stem import WordNetLemmatizer 
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier

Загрузим данные.

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Язык текста - английский.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропусков нет.

In [5]:
data['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Нетоксичные комментарии составляют около 90% данных.

Создадим функцию очистки текста, избавимся от всех знаков, кроме букв латинского алфавита.

In [6]:
def clear_text(text):
    
    return " ".join((re.sub(r'[^a-zA-Z]', ' ', text).split()))

Применим ее.

In [7]:
data['cleared'] = data['text'].apply(lambda x: clear_text(x))

In [8]:
data

,text,toxic,cleared
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,And for the second time of asking when your vi...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That is a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm theres no actual article for prost...
159569,And it looks like it was actually you who put ...,0,And it looks like it was actually you who put ...


Затем, чтобы провести более качественную лемматизацию сделаем следующее:

1. Проставим теги частей речи при помощи инструментов nltk.

In [9]:
nltk.download('averaged_perceptron_tagger') 
 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


2. Создадим функцию замены тегов nltk на теги wordnet.

In [10]:
def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None

3. Создадим функцию для лемматизации с уточненными частями речи. Будем использовать WordNetLemmatizer.

In [11]:
def pos_lemmatize(text):
    
    #проставим теги через nltk
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))
    
    #заменим их тегами wordnet
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    
    #лемматизируем
    wnl = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in wordnet_tagged: 
        if tag is None: 
            # если нет подходящего тега, то добавляем токен как есть
            lemmatized_sentence.append(word) 
        else:         
            # в противном случае используем тег для лемматизации
            lemmatized_sentence.append(wnl.lemmatize(word, tag)) 
    lemmatized_sentence = " ".join(lemmatized_sentence)
    return lemmatized_sentence

In [12]:
%%time
data['lemm_text'] = data['cleared'].apply(lambda x: pos_lemmatize(x))

CPU times: user 13min 10s, sys: 5.53 s, total: 13min 15s
Wall time: 13min 22s


In [13]:
data

,text,toxic,cleared,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...,Hey man I m really not try to edit war It s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...,You sir be my hero Any chance you remember wha...
...,...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,And for the second time of asking when your vi...,And for the second time of ask when your view ...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That is a ho...,You should be ashamed of yourself That be a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm theres no actual article for prost...,Spitzer Umm theres no actual article for prost...
159569,And it looks like it was actually you who put ...,0,And it looks like it was actually you who put ...,And it look like it be actually you who put on...


Удалим ненужные данные, сотавим только столбец с целевым признаком и лемматизированным текстом.

In [14]:
data = data.drop(['cleared', 'text'], axis=1)

# 2. Обучение

Выделим целевой признак и разделим данные на обучающую и тестовую выборку в соотношении 3:1 со стратификацией. 

In [15]:
target = data['toxic']
features = data.drop('toxic', axis=1)

features_train, features_test, target_train, target_test = train_test_split(features, 
                                        target, test_size=0.25, random_state=2, stratify = target)

Получим обучающую выборку в кодировке Юникод, загрузим список английских стоп-слов.

In [16]:
corpus_train = features_train['lemm_text'].values.astype('U') #список твитов становится обучающей выборкой

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english')) #загрузили стоп-слова

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Затем создадим счетчик TF-IDF, передадим в него английские стоп-слова, будем также искать биграммы, ограничим максимальное количество признаков 20000, ограничим минимальное вхождение n-gram в текст двумя.

Посчитаем TF-IDF для обучающей выборки.

In [17]:
%%time
count_tf_idf = TfidfVectorizer(stop_words=stopwords, ngram_range = (1,2), min_df=2, max_features=20000)
tf_idf_train = count_tf_idf.fit_transform(corpus_train)

CPU times: user 27.6 s, sys: 788 ms, total: 28.4 s
Wall time: 28.8 s


Создадим скорер F1-меры.

In [18]:
f1 = make_scorer(f1_score)

Инициализируем логистическую регрессию.

In [19]:
log_reg = LogisticRegression(n_jobs=3, solver='lbfgs', 
                           random_state=2, verbose=1)

Воспользуемся инструментом GridSearchCV и попробуем подобрать параметр, регулирующий инверсию силы регуляризации. То еость, попробуем усилить и ослабить регуляризацию.

In [20]:
%%time
param_grid_log_reg = {'C': [0.1, 1, 10 ,15]}
grid_log_reg = GridSearchCV(log_reg, 
                          param_grid_log_reg, 
                          return_train_score=True, 
                          cv=3, n_jobs=-1, scoring=f1)

grid_log_reg.fit(tf_idf_train, target_train)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   1 out of   1 | elapsed:    3.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   1 out of   1 | elapsed:    1.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   1 out of   1 | elapsed:    1.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   1 out of   1 | elapsed:    2.4s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   1 out of   1 | elapsed:    3.4s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   1 out of   1 | elapsed:    2.5s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   1 out of   1 | elapsed:   

CPU times: user 1.28 s, sys: 924 ms, total: 2.21 s
Wall time: 41.2 s


[Parallel(n_jobs=3)]: Done   1 out of   1 | elapsed:    4.2s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=3, penalty='l2',
                                          random_state=2, solver='lbfgs',
                                          tol=0.0001, verbose=1,
                                          warm_start=False),
             iid='warn', n_jobs=-1, param_grid={'C': [0.1, 1, 10, 15]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=make_scorer(f1_score), verbose=0)

In [21]:
print(f'F1-мера лучшей модели: {grid_log_reg.best_score_:.3f}')

F1-мера лучшей модели: 0.761


In [22]:
print(f'С-инверсия силы регуляризации: {grid_log_reg.best_params_}')

С-инверсия силы регуляризации: {'C': 10}


Итак, в процессе кросс-валидации мы достигли значения F1-меры в размере 0.76, при этом параметр С = 10, значит, мы ослабили регуляризацию. Вообще, это может приводить к переобучению. Протестируем модель.

Получим тестовую выборку в кодировке Юникод.

In [23]:
corpus_test = features_test['lemm_text'].values.astype('U')

/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/backend/utils.py:55: UserWarning: Failed to kill subprocesses on this platform. Pleaseinstall psutil: https://github.com/giampaolo/psutil
  warnings.warn("Failed to kill subprocesses on this platform. Please"
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/backend/utils.py:55: UserWarning: Failed to kill subprocesses on this platform. Pleaseinstall psutil: https://github.com/giampaolo/psutil
  warnings.warn("Failed to kill subprocesses on this platform. Please"
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/backend/utils.py:55: UserWarning: Failed to kill subprocesses on this platform. Pleaseinstall psutil: https://github.com/giampaolo/psutil
  warnings.warn("Failed to kill subprocesses on this platform. Please"


Посчитаем TF-IDF для тестовой выборки.

In [24]:
tf_idf_test = count_tf_idf.transform(corpus_test)

Предскажем значения тестовой выборки.

In [25]:
test_predict = grid_log_reg.best_estimator_.predict(tf_idf_test)
print(f'F1-мера на тестовой выборке: {f1_score(target_test, test_predict):.3f}')

F1-мера на тестовой выборке: 0.765


Как мы видим, переобучения не произошло, значение F1-меры даже немного выросло.

# 3. Выводы

1. Мы располагали датасетом, содержащим размеченные коментарии на токсичные и нетоксичные. Токсичных было около 10%.

2. Во время подготовки текстовых данных мы их очистили от знаков препинания, расставили теги частей речи, и по этим тегам провели более точную лемматизацию.

3. При помоща кросс-валидации подбирали лучший параметр инверсии силы регуляризации для логистической регрессии, в итоге немного ее ослабили, что, вроде бы, не привело к переобучению.

4. Значение F1-меры в 0.75 было достигнуто.

5. Посмотрим на матрицу ошибок нашей модели, полноту и точность.

In [26]:
confusion_matrix(target_test, test_predict)

array([[35401,   436],
       [ 1274,  2782]])

In [27]:
recall_score(target_test, test_predict)

0.6858974358974359

Доля истинно-положительных ответов среди всех, которым модель выдала метку 1 - около 0.69. Модель иногда пропускает "токсичные" комментарии.

In [28]:
precision_score(target_test, test_predict)

0.8645121193287756

Точность нашей модели достаточно высока - около 0.86, то есть, можно сказать, что модель, скорее "осторожничает" при оценке "токсичности" комментария. 